In [2]:
import pandas as pd
import numpy as np
from groq import Groq
import os
import pinecone
import re
import sys
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_pinecone import PineconeVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

from IPython.display import display, HTML

In [ ]:


groq_api_key = 'add-your-api-key'
client = Groq(api_key = groq_api_key)
model = "llama3-70b-8192" #llama 3


In [ ]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import login



# Login to Hugging Face
login(token="add-your-key")

# Using Meta-Llama model with a specified tokenizer

model_id = "meta-llama/Meta-Llama-3-70B-Instruct" 

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Function to calculate the token length
def token_len(text):
    tokens = tokenizer.encode(text)#
    return len(tokens)

# Embedding each chunk using a SentenceTransformer model
from sentence_transformers import SentenceTransformer
embedding_function = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2") #full



In [5]:
def remove_links(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Remove URLs from the text
    text_without_links = re.sub(url_pattern, '', text)

    return text_without_links

def remove_line_breaks_and_links(text):
    # Remove line breaks
    text_without_line_breaks = re.sub(r'\n{2,}', '\n', text)
    remove_leading_indent = re.sub(r'\n\s+', '\n', text_without_line_breaks)  # Remove leading indentation
    
    # Remove links
    text_without_links = remove_links(remove_leading_indent)

    return text_without_links

In [ ]:
# #Gender
df = pd.read_csv('/Data/target_gender_ads.csv', index_col=False) #gender


#Age
#df = pd.read_csv('/Data/target_age_ads.csv', index_col=False) #age

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.reset_index(drop=True)
print(df.columns, df.shape)

In [ ]:
 # Initialize an empty list to store the assistant's replies
assistant_replies = []

In [ ]:
def chat_completion(client, model, user_question):
    try:
        chat_completion = client.chat.completions.create(
            messages = [
                
                {
                    "role": "user",
                    "content":  user_question ,
                }
            ],
            model = model
        )
        
        response = chat_completion.choices[0].message.content
        return response

    except groq.RateLimitError as e :
        # Handle rate limit exceeded error by waiting for a fixed time (e.g., 60 seconds)
        print("Rate limit exceeded. Waiting for 10 seconds...")
        time.sleep(10)
        return chat_completion(client, model, user_question)  # Retry the request after waiting



In [ ]:
for index, row in df.iterrows():
    text = row['ad_creative_body']
    cleaned_text = remove_line_breaks_and_links(text).strip()
    
    #gender
    user_message = "Ad: "+cleaned_text+"\n Report who can be targeted based on gender: male vs female by the provided ad. Choose the most probable option from the gender (either male or female). Don't generate neutral or both gender answers. Provide a brief explanation with reasoning in 20 words. \n Gender: ? \n Explanation: ?"
   #age group
    #user_message = "Ad: "+cleaned_text+"\n Report who can be targeted based on the following four age groups: Young adults (18-24), Early working age (25-44), Late working age (45-64), and Senior citizens (65+) by the provided ad. Choose the most probable option from the age groups. Don't generate neutral or multiple age group answers. Provide a brief explanation with the reasoning in 20 words.  \n Age group: ? \n Explanation: ?"
    
    
    assistant_reply = chat_completion(client, model, user_message)
    
    
    # Append the assistant's reply to the list
    assistant_replies.append(assistant_reply)
    
# Add the assistant's replies to the DataFrame
df['llama_reply'] = assistant_replies


In [ ]:
# Print or save the updated DataFrame with assistant's replies
df.to_csv('/Data/gender_pred_llama.csv')